<a href="https://colab.research.google.com/github/Winter-Shade/Text-To-SQL-LLM/blob/main/LLM_project_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 whi

In [ ]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("gretelai/synthetic_text_to_sql")

train_test_split = dataset["train"].train_test_split(test_size=0.2, seed=42)

dataset = DatasetDict({
    "train": train_test_split["train"],
    "validation": train_test_split["test"],
    "test": dataset["test"]
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 80000
    })
    validation: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 20000
    })
    test: Dataset({
        features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation'],
        num_rows: 5851
    })
})

In [ ]:
import torch
from torch.utils.data import Dataset

class SQLDataset(Dataset):

  def __init__(self, sql_prompts, sql_queries):
    self.sql_prompts = sql_prompts
    self.sql_queries = sql_queries

  def __len__(self):
    return len(self.sql_prompts)

  def __getitem__(self, index):
    return self.sql_prompts[index], self.sql_queries[index]

In [ ]:
train_sql_prompts = dataset["train"]["sql_prompt"]
train_sql_queries = dataset["train"]["sql"]

train_dataset = SQLDataset(train_sql_prompts, train_sql_queries)

In [ ]:
validation_sql_prompts = dataset["validation"]["sql_prompt"]
validation_sql_queries = dataset["validation"]["sql"]

val_dataset = SQLDataset(validation_sql_prompts, validation_sql_queries)

In [ ]:
test_sql_prompts = dataset["test"]["sql_prompt"]
test_sql_queries = dataset["test"]["sql"]

test_dataset = SQLDataset(test_sql_prompts, test_sql_queries)

In [ ]:
train_dataset[1]

("List all urban gardens in 'North America' with their respective sizes.",
 "SELECT name, size FROM gardens WHERE location LIKE 'North America';")

In [ ]:
train_dataset.__getitem__(34)

('Which countries source ingredients for a specific cosmetic product?',
 "SELECT pc.product_name, i.country FROM ingredient_sourcing i JOIN product_catalog pc ON i.product_id = pc.product_id WHERE pc.product_name = 'Example Product';")